In [470]:
#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [471]:
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import plotly.express as px
import lmfit as lm
%matplotlib inline

In [472]:
# setting display option for pandas for columns
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [473]:
response = requests.get('https://api.rootnet.in/covid19-in/stats/history')
covid_data = response.json()['data']
keys = ['day', 'total','discharged', 'deaths']

covid_india = pd.DataFrame([[d.get('day'), 
                                  d['summary'].get('total'), 
                                  d['summary'].get('discharged'), 
                                  d['summary'].get('deaths')] 
                                 for d in covid_data],
                    columns=keys)

In [474]:
covid_india.head()

,day,total,discharged,deaths
0,2020-03-10,47,0,0
1,2020-03-11,60,0,0
2,2020-03-12,73,0,0
3,2020-03-13,82,10,2
4,2020-03-14,84,10,2


In [475]:
covid_india.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   day         534 non-null    object
 1   total       534 non-null    int64 
 2   discharged  534 non-null    int64 
 3   deaths      534 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 16.8+ KB


In [476]:
covid_india['Date'] = pd.to_datetime(covid_india['day'], infer_datetime_format=True)
covid_india.drop(columns='day',inplace=True)

In [477]:
vaccine_data = pd.read_csv("vaccine_doses_statewise_v2.csv")
vaccine_data.head()

,Vaccinated As of,State,First Dose Administered,Second Dose Administered,Total Doses Administered
0,16/01/2021,Andaman and Nicobar Islands,225,0,225
1,16/01/2021,Andhra Pradesh,18412,0,18412
2,16/01/2021,Arunachal Pradesh,829,0,829
3,16/01/2021,Assam,3528,0,3528
4,16/01/2021,Bihar,18169,0,18169


In [478]:
vaccine_data.rename(columns = {'Vaccinated As of':'Date'}, inplace=True)
#Converting date ibject type to datetime
vaccine_data.Date = pd.to_datetime(vaccine_data.Date, infer_datetime_format=True)
vaccine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9120 entries, 0 to 9119
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      9120 non-null   datetime64[ns]
 1   State                     9120 non-null   object        
 2   First Dose Administered   9120 non-null   int64         
 3   Second Dose Administered  9120 non-null   int64         
 4   Total Doses Administered  9120 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 356.4+ KB


In [479]:
vaccine_data.drop(columns=['First Dose Administered', 'Total Doses Administered'], inplace=True)
vaccine_data.head()

,Date,State,Second Dose Administered
0,2021-01-16,Andaman and Nicobar Islands,0
1,2021-01-16,Andhra Pradesh,0
2,2021-01-16,Arunachal Pradesh,0
3,2021-01-16,Assam,0
4,2021-01-16,Bihar,0


In [480]:
vaccine_data_india = vaccine_data[vaccine_data.State == 'Total'].reset_index().drop(columns = 'index')

In [481]:
covid_india_data = pd.merge(left= covid_india, right= vaccine_data_india, how='left', on = 'Date')

In [482]:
covid_india_data.drop(columns=['State', 'Date'], inplace=True)
covid_india_data.rename(columns = {'total':'Infected', 'discharged':'Recovered', 'deaths':'Deceased', 'Second Dose Administered':'Vaccinated'}, inplace=True)
covid_india_data.fillna(0, inplace=True)
covid_india_data.head()

,Infected,Recovered,Deceased,Vaccinated
0,47,0,0,0.0
1,60,0,0,0.0
2,73,0,0,0.0
3,82,10,2,0.0
4,84,10,2,0.0


In [483]:
covid_india_data.shape

(534, 4)

In [484]:
covid_india_data[covid_india_data.Vaccinated > 0].head()

,Infected,Recovered,Deceased,Vaccinated
340,10892746,10600625,155550,23628.0
341,10904940,10611731,155642,24561.0
342,10916589,10621220,155732,160291.0
343,10925710,10633025,155813,276377.0
344,10937320,10644858,155913,369167.0


In [485]:
def SEIRD(y, t,  beta, sigma, gamma, mu):
    S, E, I, R, D = y
    N = S + E + I + R + D
    dSdt = - beta * S * I / N
    dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I - mu*I
    dRdt = gamma*I
    dDdt = mu*I
    return dSdt, dEdt, dIdt, dRdt, dDdt

In [486]:
def odesolver_SEIRD(t, initial_state, params):
    S0, E0, I0, R0, D0, V0 = initial_state
    beta  = params['beta0'].value
    gamma = params['gamma0'].value 
    sigma = params['sigma0'].value 
    mu = params['mu0'].value
    res = odeint(SEIRD, [S0, E0, I0, R0, D0], t, args = (beta, sigma, gamma, mu))
    return res

In [487]:
def residuals_SEIRD(params, t, state, data):
    expected = odesolver_SEIRD(t, state, params)
    observed = data
    return (expected[:,2:5] - observed).ravel()

In [488]:
N = 1366400000
E0 = 1000
I0, R0 = 47, 0
D0 = 0
V0 = 0
S0 = N - I0 - R0 - E0 - D0 - V0
initial_state = [S0, E0, I0, R0, D0, V0]

params = lm.Parameters()
params.add('beta0', value=1.14, min=0, max=10)
params.add('gamma0', value=0.02, min=0, max=10)
params.add('sigma0', value=0.02, min=0, max=10)
params.add('mu0', value=0.01, min=0, max=10)

In [489]:
d = int(0.5*340)
tspan_train = np.arange(0,d,1)

data_india_before_vacc = covid_india_data.loc[0:d-1, ['Infected', 'Recovered', 'Deceased']].values


In [490]:
result1 = lm.minimize(residuals_SEIRD, params, args = (tspan_train, initial_state, data_india_before_vacc), method = 'leastsq')
result1.params.pretty_print()

Name       Value      Min      Max   Stderr     Vary     Expr Brute_Step
beta0    0.08659        0       10 0.006776     True     None     None
gamma0   0.03574        0       10 0.001029     True     None     None
mu0     0.001016        0       10 0.0006156     True     None     None
sigma0     6.063        0       10    43.46     True     None     None


In [491]:
beta1 = result1.params['beta0'].value
gamma1 = result1.params['gamma0'].value
sigma1 = result1.params['sigma0'].value
mu1 = result1.params['mu0'].value

In [492]:
tspan = np.arange(0,340,1)
ret = odeint(SEIRD, [S0, E0, I0, R0, D0], tspan, args = (beta1, sigma1, gamma1, mu1))

In [493]:
S, E, I, R, D = ret.T

In [494]:
expected_data = pd.DataFrame({'Days':tspan,
                                'Susceptible':S,
                                'Exposed' : E,
                                'Infected':I,
                                'Recovered':R,
                                'Deceased':D})

In [495]:
expected_data.head()

,Days,Susceptible,Exposed,Infected,Recovered,Deceased
0,0,1.366399e+09,1000.000000,47.000000,0.000000,0.000000
1,1,1.366399e+09,17.350117,1074.077824,31.859446,0.905430
2,2,1.366399e+09,16.017619,1130.363041,71.266254,2.025352
3,3,1.366399e+09,16.819643,1187.299158,112.670504,3.202040
4,4,1.366399e+09,17.666763,1247.098226,156.160132,4.437994


In [496]:
expected_data.shape

(340, 6)

In [497]:
fig = px.line(expected_data, x='Days', y = ['Susceptible','Infected','Recovered','Deceased'])
fig.show()

In [498]:
def SEIRDV(y, t, beta, sigma, gamma, nu, mu, alpha):
    S, E, I, R, D, V = y
    N = S + E + I + R + D + V
    dSdt = - beta * S * I / N - alpha * nu * S
    dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I - mu*I
    dRdt = gamma*I
    dDdt = mu*I
    dVdt = alpha * nu * S
    return dSdt, dEdt, dIdt, dRdt, dDdt, dVdt

In [499]:
def odesolver_SEIRDV(t, intermediate_state, params):
    S1, E1, I1, R1, D1, V1 = intermediate_state
    beta  = params['beta1'].value
    gamma = params['gamma1'].value 
    sigma = params['sigma1'].value 
    mu = params['mu1'].value
    nu = params['nu'].value
    alpha = params['alpha'].value
    res = odeint(SEIRDV, [S1, E1, I1, R1, D1, V1], t, args = (beta, sigma, gamma, mu, nu, alpha))
    return res

In [500]:
def residuals_SEIRDV(params, t, state, data):
    expected = odesolver_SEIRDV(t, state, params)
    observed = data
    return (expected[:,2:6] - observed).ravel()

In [501]:
E1 = expected_data.iloc[339,2]
I1, R1 = expected_data.iloc[339,3], expected_data.iloc[339,4]
D1 = expected_data.iloc[339,5]
V0 = 0
S1 = N - I1 - R1 - E1 - D1 - V0
intermediate_state = [S1, E1, I1, R1, D1, V0]

params = lm.Parameters()
params.add('beta1', value=beta1, min=0, max=10)
params.add('gamma1', value=gamma1, min=0, max=10)
params.add('sigma1', value=sigma1, min=0, max=10)
params.add('mu1', value=mu1, min=0, max=10)
params.add('nu', value =0.01, min=0, max =1)
params.add('alpha', value=0.7, min = 0, max=1)

In [502]:
d = int(0.5*(528-340))
tspan_train = np.arange(0,d,1)

data_india_after_vacc = covid_india_data.loc[340:340+d-1,:].values

In [503]:
result2 = lm.minimize(residuals_SEIRDV, params, args = (tspan_train, intermediate_state, data_india_after_vacc), method = 'leastsq')
result2.params.pretty_print()

Name       Value      Min      Max   Stderr     Vary     Expr Brute_Step
alpha     0.8098        0        1     None     True     None     None
beta1    0.04538        0       10     None     True     None     None
gamma1  7.33e-10        0       10     None     True     None     None
mu1     0.001742        0       10     None     True     None     None
nu       0.00865        0        1     None     True     None     None
sigma1  1.533e-10        0       10     None     True     None     None


In [504]:
beta = result2.params['beta1'].value
gamma = result2.params['gamma1'].value
sigma = result2.params['sigma1'].value
mu = result2.params['mu1'].value
nu = result2.params['nu'].value
alpha = result2.params['nu'].value

In [505]:
tspan = np.arange(0,188,1)
ret2 = odeint(SEIRDV, [S1, E1, I1, R1, D1, V0], tspan, args = (beta, sigma, gamma, mu,nu, alpha))

In [506]:
S,E,I,R,D,V = ret2.T

In [507]:
expected_data2 = pd.DataFrame({'Days':tspan,
                                'Susceptible':S,
                                'Exposed' : E,
                                'Infected':I,
                                'Recovered':R,
                                'Deceased':D,
                                'Vaccinated':V})

In [508]:
fig = px.line(expected_data2, x='Days', y = ['Susceptible','Infected','Recovered','Deceased'])
fig.show()

In [509]:
expected_data2['Days'] = expected_data2['Days']+340

In [510]:
expected_data = expected_data.append(expected_data2).fillna(0)

In [511]:
fig = px.line(expected_data, x='Days', y = ['Susceptible','Infected','Recovered','Deceased'])
fig.show()

In [512]:
global_data_infected = pd.read_csv("time_series_covid19_confirmed_global.csv")
global_data_infected.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21,6/6/21,6/

In [513]:
global_data_recovered = pd.read_csv("time_series_covid19_recovered_global.csv")
global_data_recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21,6/6/21,6/

In [514]:
global_data_deceased = pd.read_csv("time_series_covid19_deaths_global.csv")
global_data_deceased.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21,6/6/21,6/

In [515]:
us_vaccine_data = pd.read_csv("us_state_vaccinations.csv")
us_vaccine_data.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,12-01-2021,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,13-01-2021,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,14-01-2021,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,15-01-2021,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,16-01-2021,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7498.0,1529.0,NaN


In [516]:
global_data_infected.drop(columns=['Province/State', 'Lat', 'Long'], inplace=True)
global_data_recovered.drop(columns=['Province/State', 'Lat', 'Long'], inplace=True)
global_data_deceased.drop(columns=['Province/State', 'Lat', 'Long'], inplace=True)

In [517]:
global_data_infected.columns[1:]

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '9/15/21', '9/16/21', '9/17/21', '9/18/21', '9/19/21', '9/20/21',
       '9/21/21', '9/22/21', '9/23/21', '9/24/21'],
      dtype='object', length=612)

In [518]:
us_infected_data = pd.melt(global_data_infected[global_data_infected['Country/Region'] == 'US'], id_vars='Country/Region',
                           var_name='Date', value_name='Infected').drop(columns='Country/Region')
us_recovered_data = pd.melt(global_data_recovered[global_data_recovered['Country/Region'] == 'US'], id_vars='Country/Region',
                           var_name='Date', value_name='Recovered').drop(columns='Country/Region')
us_deceased_data = pd.melt(global_data_deceased[global_data_deceased['Country/Region'] == 'US'], id_vars='Country/Region',
                           var_name='Date', value_name='Deceased').drop(columns='Country/Region')

In [519]:
us_infected_data.head()

,Date,Infected
0,1/22/20,1
1,1/23/20,1
2,1/24/20,2
3,1/25/20,2
4,1/26/20,5


In [520]:
us_recovered_data.head()

,Date,Recovered
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0
3,1/25/20,0
4,1/26/20,0


In [521]:
us_deceased_data.head()

,Date,Deceased
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0
3,1/25/20,0
4,1/26/20,0


In [522]:
us_covid_data = pd.concat([us_infected_data, us_recovered_data['Recovered'], us_deceased_data['Deceased']], axis=1)
us_covid_data.head()

,Date,Infected,Recovered,Deceased
0,1/22/20,1,0,0
1,1/23/20,1,0,0
2,1/24/20,2,0,0
3,1/25/20,2,0,0
4,1/26/20,5,0,0


In [523]:
us_vaccine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16663 entries, 0 to 16662
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 16663 non-null  object 
 1   location                             16663 non-null  object 
 2   total_vaccinations                   16133 non-null  float64
 3   total_distributed                    15875 non-null  float64
 4   people_vaccinated                    15863 non-null  float64
 5   people_fully_vaccinated_per_hundred  15007 non-null  float64
 6   total_vaccinations_per_hundred       15109 non-null  float64
 7   people_fully_vaccinated              15781 non-null  float64
 8   people_vaccinated_per_hundred        15084 non-null  float64
 9   distributed_per_hundred              15096 non-null  float64
 10  daily_vaccinations_raw               15736 non-null  float64
 11  daily_vaccinations          

In [524]:
us_vaccine_data = us_vaccine_data[['date', 'location','people_fully_vaccinated']]
us_vaccine_data.interpolate(method='linear', axis=0, inplace=True)
us_vaccine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16663 entries, 0 to 16662
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     16663 non-null  object 
 1   location                 16663 non-null  object 
 2   people_fully_vaccinated  16663 non-null  float64
dtypes: float64(1), object(2)
memory usage: 390.7+ KB


In [525]:
us_vaccine_data = us_vaccine_data[us_vaccine_data.location == 'United States'].drop(columns='location').rename(columns={'people_fully_vaccinated' : 'Vaccinated', 'date':'Date'})

In [526]:
us_vaccine_data.head()

,Date,Vaccinated
14080,20-12-2020,1.413632e+07
14081,21-12-2020,1.355571e+07
14082,22-12-2020,1.297509e+07
14083,23-12-2020,1.239448e+07
14084,24-12-2020,1.181387e+07


In [527]:
us_covid_data['Date'] = pd.to_datetime(us_covid_data['Date'], infer_datetime_format=True)
us_vaccine_data['Date'] = pd.to_datetime(us_vaccine_data['Date'], infer_datetime_format=True)
us_covid_data = pd.merge(left = us_covid_data, right= us_vaccine_data, how='left', on='Date').fillna(0)

In [528]:
us_covid_data.head()

,Date,Infected,Recovered,Deceased,Vaccinated
0,2020-01-22,1,0,0,0.0
1,2020-01-23,1,0,0,0.0
2,2020-01-24,2,0,0,0.0
3,2020-01-25,2,0,0,0.0
4,2020-01-26,5,0,0,0.0


In [529]:
us_covid_data = us_covid_data[us_covid_data.Date >= '2020-03-01']
us_covid_data.head()

,Date,Infected,Recovered,Deceased,Vaccinated
39,2020-03-01,32,7,1,0.0
40,2020-03-02,55,7,6,0.0
41,2020-03-03,74,7,7,0.0
42,2020-03-04,107,7,11,0.0
43,2020-03-05,184,7,12,0.0


In [531]:
us_covid_data[us_covid_data.Vaccinated > 0]

,Date,Infected,Recovered,Deceased,Vaccinated
333,2020-12-20,17999535,0,322756,1.413632e+07
334,2020-12-21,18190836,0,324656,1.355571e+07
335,2020-12-22,18392072,0,328055,1.297509e+07
336,2020-12-23,18617481,0,331478,1.239448e+07
337,2020-12-24,18822716,0,334374,1.181387e+07
338,2020-12-25,18930669,0,335832,1.123326e+07
339,2020-12-26,19149189,0,337723,1.065264e+07
340,2020-12-27,19303667,0,339160,1.007203e+07
341,2020-12-28,19471670,0,341162,9.491419e+06
342,2020-12-29,19672879,0,344781,8.910806e+06
